<a href="https://colab.research.google.com/github/HaeMinJ/EconomicDataScience/blob/main/UpbitSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
INITIAL_MONEY = 100000
CURRENT_UUID = 0
UPBIT_TRADE_FEE = 0.0005

In [7]:
import pyupbit
import math
import numpy as np
import datetime

In [6]:
print(pyupbit.get_tickers(fiat="KRW"))

['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC', 'KRW-XRP', 'KRW-ETC', 'KRW-OMG', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-REP', 'KRW-ADA', 'KRW-SBD', 'KRW-POWR', 'KRW-BTG', 'KRW-ICX', 'KRW-EOS', 'KRW-TRX', 'KRW-SC', 'KRW-ONT', 'KRW-ZIL', 'KRW-POLY', 'KRW-ZRX', 'KRW-LOOM', 'KRW-BCH', 'KRW-BAT', 'KRW-IOST', 'KRW-RFR', 'KRW-CVC', 'KRW-IQ', 'KRW-IOTA', 'KRW-MFT', 'KRW-ONG', 'KRW-GAS', 'KRW-UPP', 'KRW-ELF', 'KRW-KNC', 'KRW-BSV', 'KRW-THETA', 'KRW-QKC', 'KRW-BTT', 'KRW-MOC', 'KRW-ENJ', 'KRW-TFUEL', 'KRW-MANA', 'KRW-ANKR', 'KRW-AERGO', 'KRW-ATOM', 'KRW-TT', 'KRW-CRE', 'KRW-MBL', 'KRW-WAXP', 'KRW-HBAR', 'KRW-MED', 'KRW-MLK', 'KRW-STPT', 'KRW-ORBS', 'KRW-VET', 'KRW-CHZ', 'KRW-STMX', 'KRW-DKA', 'KRW-HIVE', 'KRW-KAVA', 'KRW-AHT', 'KRW-LINK', 'KRW-XTZ', 'KRW-BORA', 'KRW-JST', 'KRW-CRO', 'KRW-TON', 'KRW-SXP', 'KRW-HUNT', 'KRW-PLA', 'KRW-DOT', 'KRW-SRM', 'KRW-MVL', 'KRW-STRAX', 'KRW-AQT', 'KRW-GLM', 

In [ ]:
class Balance:
  def __init__(self, currency, balance, locked, avg_buy_price, avg_buy_price_modified=False, unit_currency='KRW'):
    self.currency = currency
    self.balance = balance
    self.locked = locked
    self.avg_buy_price = avg_buy_price
    self.avg_buy_price_modified = avg_buy_price_modified
    self.unit_currency = unit_currency

In [ ]:
class User:
  def __init__(self, name, KRW):
    self.name = name
    self.balances = []
    self.orders = []
    self.balances.append(Balance('KRW', KRW, 0.0, 0))

In [ ]:
class Order:
  def __init__(self, side, ord_type, price, market, volume):
    self.uuid = ++CURRENT_UUID
    self.side = side
    self.ord_type = ord_type
    self.price = price
    self.state = 'wait'
    self.market = market
    self.created_at = datetime.datetime.now()
    self.volume = volume
    self.remaining_volume = volume
    self.reserved_fee = price*volume*UPBIT_TRADE_FEE
    self.remaining_fee = self.reserved_fee
    self.paid_fee = 0.0
    self.locked = price*volume+self.remaining_fee
    self.excuted_volume = 0.0
    self.trades_count = 0

In [ ]:
class VUpbit:
  orderQueue = []
  def __init__(self, access, secret):
    self.user = access
    self.balances = self.user.balances
    self.orders = self.user.orders
    #Order-Queue 
  def get_balances(self):
    return self.balances
  def sell_limit_order(self, ticker, price, volume, contain_req=False):
    order = Order('bid', 'limit',price,ticker,volume)
    self.orderQueue.append(order)
    self.orders.append(order)
  def buy_limit_order(self, ticker, price, volume, contain_req=False):
    order = Order('ask', 'limit',price,ticker,volume)
    self.orderQueue.append(order)
    self.orders.append(order)
    print("Buy %s for %d"%(ticker,price,volume))
    #지정가 매수매도.
  def buy_market_order(self, ticker, price, contain_req=False):
    market_price = pyupbit.get_current_price(ticker)
    volume = price/market_price
    order = Order('ask', 'limit',market_price,ticker, volume )
    self.orderQueue.append(order)
    self.orders.append(order)
    print("Buy %s for %d"%(ticker,price))

  def sell_market_order(self, ticker, volume, contain_req=False):
    market_price = pyupbit.get_current_price(ticker)
    order = Order('bid', 'limit',market_price, ticker, volume )
    self.orderQueue.append(order)
    self.orders.append(order)
    print("Sell %d %s"%(volume, ticker))
    #시장가 매수매도.
  def cancel_order(self, uuid, contain_req=False):
    
    print("Cancle Order %s"%(uuid))
  def get_order(self, ticker_or_uuid, state='wait', kind='normal', contain_req=False):
    return self.orders

In [ ]:
user = User(name="HaeMinJ", KRW = INITIAL_MONEY )
user.sell_limit_order()